In [36]:
# imports

import torch
from transformers import Trainer, MT5Model, AutoTokenizer, pipeline, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, MT5ForConditionalGeneration
from torch.utils.data import DataLoader
import json
from datasets import Dataset
from sklearn.model_selection import train_test_split
import random

device = torch.device('cpu')


In [37]:
# loading the data

all_data = []

with open('data.json', 'r') as data:
    data = json.load(data)
    for datapoint in data:
        entry = {
            "original_headline": datapoint['original']['headline'],
            "original_article": datapoint['original']['article'],
            "target_headline": datapoint['simplified']['headline'],
            "target_article": datapoint['simplified']['article']
        }
        all_data.append(entry)

# splitting the data

training_data, test_data = train_test_split(all_data, test_size=0.2)        

# creating datasets 

train_dataset = Dataset.from_list(training_data)
test_dataset = Dataset.from_list(test_data)


In [38]:
# instantiating tokenizer

tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

c:\Users\annin\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [39]:
# define preprocessing function

# TO DO! CHECK HOW TO ALSO INCLUDE THE ARTICLE HEADLINE

def preprocess_function(data):
    inputs = [article for article in data['original_article']]
    model_inputs = tokenizer(inputs, max_length=500, truncation=True, padding=True, return_tensors='pt')
    labels = tokenizer(text_target=data['target_article'], max_length=200, truncation=True, padding=True, return_tensors='pt')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [40]:
# apply preprocessing function to the dataset

tokenized_training_data = train_dataset.map(preprocess_function, batched=True)
tokenized_test_data = test_dataset.map(preprocess_function, batched=True)
print(tokenized_test_data[0])


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

{'original_headline': 'Claudia Sheinbaum blir Mexikos första kvinnliga president', 'original_article': 'Ett val kantat av dödligt våld har nått sitt slut. Claudia Sheinbaum blir Mexikos första kvinnliga president. Det första officiella preliminära resultatet pekade på en överlägsen seger för Claudia Sheinbaum, som tillhör det regerande partiet Morena. Enligt beräkningar från Mexikos internationella valinstitut har Sheinbaum fått drygt 60 procent av rösterna, samtidigt som huvudmotståndaren Xóchitl Gálvez har fått cirka 25 procent av rösterna. Morena, som redan tidigare utropat seger, kommer att fortsätta med en enkel majoritet i landets kongress, säger partichefen Mario Delgado under en nationell tv-sändning. Största och farligaste valet Mexikos största val någonsin har också varit det mest våldsamma i modern historia, med dödandet av 38 kandidater. Våldet har väckt oro i landet att demokratin ligger under hot av de stridande drogkartellerna. Att konfrontera och minska den organiserade

In [41]:
# creating the dataloader 

train_dataloader = DataLoader(tokenized_training_data, batch_size=4, shuffle=True)
test_dataloader = DataLoader(tokenized_test_data, batch_size=4, shuffle=True)

In [63]:
# instantiating the model and data collator
model =  MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [64]:
# defining training arguments

training_args = Seq2SeqTrainingArguments(
    output_dir="L2-swedish-news-article-simplifier-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_strategy="epoch",
    overwrite_output_dir=True,
    num_train_epochs=4,
    logging_steps=10,
    load_best_model_at_end=True,
    predict_with_generate=True,
    bf16=True, #change to bf16=True for XPU
    push_to_hub=False,
)

In [65]:
# defining the trainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_data,
    eval_dataset=tokenized_test_data,
    data_collator=data_collator,
    tokenizer=tokenizer
    #compute_metrics=compute_metrics,
)

In [66]:
# fine-tuning the model:

trainer.train()

  0%|          | 0/40 [00:00<?, ?it/s]

{'loss': 24.3882, 'grad_norm': 6313.2724609375, 'learning_rate': 1.5000000000000002e-05, 'epoch': 1.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 18.147262573242188, 'eval_runtime': 20.3563, 'eval_samples_per_second': 0.491, 'eval_steps_per_second': 0.147, 'epoch': 1.0}
{'loss': 24.028, 'grad_norm': 4591.36669921875, 'learning_rate': 1e-05, 'epoch': 2.0}


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 17.36973762512207, 'eval_runtime': 17.7101, 'eval_samples_per_second': 0.565, 'eval_steps_per_second': 0.169, 'epoch': 4.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


{'train_runtime': 1050.7211, 'train_samples_per_second': 0.152, 'train_steps_per_second': 0.038, 'train_loss': 23.367663192749024, 'epoch': 4.0}


TrainOutput(global_step=40, training_loss=23.367663192749024, metrics={'train_runtime': 1050.7211, 'train_samples_per_second': 0.152, 'train_steps_per_second': 0.038, 'total_flos': 82617139200000.0, 'train_loss': 23.367663192749024, 'epoch': 4.0})

In [67]:
# checking the outcome with pipeline

news_simplifier = pipeline("text2text-generation", model='./L2-swedish-news-article-simplifier-model/checkpoint-40')
news_simplifier("headline: Ung man hittad död nära fotbollsplan i Borås – misstänkt mord article : Mitt under en pojklagsmatch hittades en okontaktbar person intill fotbollsplanen i Byttorp i Borås på lördagen. Polis larmades till platsen och matchen avbröts. Vid lunchtid meddelade polisen att det rör sig om en man i 20-årsåldern som hittats död. Händelsen utreds som misstänkt mord. Larmet kom in till polisen vid 9.30-tiden på lördagen efter att allmänheten larmat om en okontaktbar person liggandes på marken utomhus i Byttorp i Borås kommun. Personen hittades intill en idrottsanläggning, enligt polisen. – De har spelat fotboll där men vi har avbrutit den så att vi kan jobba ostört, sade Jens Andersson, presstalesperson i polisregion Väst, under förmiddagen. Polisen om misstänkta mordet i Byttorp: Okänd dödsorsak Strax efter klockan 12 bekräftade polisen på sin hemsida att personen som hittats är en man i 20-årsåldern, och att han är avliden. Dödsorsaken är för närvarande okänd och det finns i nuläget ingen misstänkt person i ärendet. – När vi kommer dit strax innan tio är mannen avliden. Just nu vet vi inte hur han har dött och vi kan inte utesluta att han utsatts för brott. Han är identifierad och anhöriga är underrättade, säger Jens Andersson och fortsätter: – Vi kommer vara kvar på platsen så länge det behövs. Just nu samlar vi pusselbitar och försöker skapa oss en bild av vad mannen gjort de sista timmarna. Polisen har spärrat av området i Byttorp, där de även jobbar med att höra personer på plats och försöker skapa sig en bild av händelsen. Även ambulans skickades till plats under förmiddagen, skriver polisen på sin hemsida men är i övrigt förtegen om detaljer kring händelsen. Klubbens ordförande: ”Flyttar dagens matcher” Idrottsanläggningen Byttorps IP är stängt tills vidare. – Vi kommer att flytta dagens matcher, vi hade en match i dag, säger Hanseman Samuelsson, ordförande för Byttorps IP. David Kryssman, klubbchef för ett av lagen som spelade på idrottsplatsen, berättar att laget lånat anläggningen då klubbens egna renoveras. Klubbchefen var själv inte på plats under matchen, men har varit i kontakt med berörda spelare, ledare och föräldrar. – De är chockade såklart. Men samtidigt är det svårt att ta in en sådan obehaglig händelse. Det blir väldigt nära och många av killarna är unga och har aldrig sett en skadad människa innan.")

c:\Users\annin\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
c:\Users\annin\anaconda3\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': '<extra_id_0> i Byttorp'}]